<a href="https://colab.research.google.com/github/yanisamrouche/data_science/blob/main/TP6_synthese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Master Informatique AMU, ISD, exercice de synthèse (TP6)


Voici un exercice vous permettant de synthétiser quelques unes de vos connaissances expérimentales en matière de classification à partir de données. Le code python doit être le plus concis possible.

Etant donné le no-free-lunch theorem, nous vous confions la tâche de déterminer le meilleur modèle pour réaliser une tâche de régression transformée en tâche de classification, avec le test statistique qui garantit ce choix avec grande confiance.

Remettre ce fichier ipynb complété. 


### Préparation des données
Nous allons utiliser un jeu de données réel - tiré de *Tsanas & Xifara : Accurate quantitative estimation of energy performance of residential buildings using statistical machine learning tools, Energy and Buildings, Vol. 49, pp. 560-567, 2012* - disponible sur Ametice (dataenergy.csv).

Les 8 premières colonnes correspondent aux attributs descriptifs et les deux dernières, aux charges de chauffage et de climatisation (dans cet ordre).
Pour les utiliser en Python, vous pourrez vous servir du code suivant :

In [2]:
import numpy as np
data = np.loadtxt("./dataenergy.csv")
X = data[:,:-2]
Y = data[:,-2:]
Yheat = Y[:,0]
Ycool = Y[:,1]

Le problème initial, tel que présenté ici, est un problème de régression multi-tâches. Nous allons le simplifier en le transformant en un problème de classification dont la classe sera le niveau de charge de chauffage : par une méthode de clustering, on veut répartir les charges de chauffage en 3 classes : faibles, moyennes, élevées.

A vous de jouer :

In [6]:
from sklearn.cluster import KMeans
# La suite ? il s'agit de définir un classifieur du k-means avec k=3 
# et d'utiliser la méthode 'fit' sur lensemble des etiquettes de chauffage Yheat
# Attention : les Y sont des vecteurs et les classifieurs sklearn ont besoin d'array :
# il faut les reshaper : Yheat_vector = Yheat.reshape(-1,1)
# Après apprentissage du kmeans, les classes des données utilisées sont stockées dans mon_classifieur.labels_
Yheat_vector = Yheat.reshape(-1,1)
kmeans = KMeans(n_clusters=3, random_state=0).fit(Yheat_vector)
y = kmeans.labels_
print(y)

[1 1 1 1 2 2 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 0 0 0 0
 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 2 2 2 0 0 0 0 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 0 0 0 0 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 0 0 0 0 2 2 2 2
 2 2 2 2 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 2 2 2 0 0 0 0 2 2 2 2 2 

### Apprentissage des modèles
Nous voulons comparer deux méthodes de classification supervisée :
1. Les k-plus proches voisins (*KNeighborsClassifier* de la classe *sklearn.neighbors*, hyperparamètre à régler : *n_neighbors*)
2. Les arbres de décision  (*DecisionTreeClassifier* de la classe *sklearn.tree*, hyperparamètre à régler : *max_depth*)

Ecrivez le code permettant de :
1. Séparer les données en un échantillon d'apprentissage et un échantillon de test (60/40)
2. Sélectionner les meilleurs valeurs des hyper-paramètres sur l'échantillon d'apprentissage par validation croisée en utilisant 10 folders (pour optimisation de l'erreur)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.40)
dic_param = {'n_neighbors':[3, 5, 7, 9, 11, 13, 15,20],'p':   [0.5, 1, 2, 5, 8, 12]}
select_hppmt = GridSearchCV(KNeighborsClassifier(), dic_param, cv=10)
select_hppmt.fit(X_train, y_train)
print(select_hppmt.best_params_)
#print(select_hppmt.cv_results_)
# A vous

{'n_neighbors': 3, 'p': 1}


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
80 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py", line 198, in fit
    return self._fit(X, y)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_base.py", line 437, in _fit
    self._check_algorithm_metric()
  File "/usr/local/lib/python3.7/dist-packages/sklearn/nei

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.40)
dic_param2 = {'criterion' : ['gini', 'entropy'],
                    'max_depth' : [i for i in range(1,21)],
                   'max_leaf_nodes' : [i for i in range(10,211,20)]}
select_hppmt2 = GridSearchCV(DecisionTreeClassifier(), dic_param2, cv=10)
select_hppmt2.fit(X_train, y_train)
print(select_hppmt2.best_params_)

{'criterion': 'gini', 'max_depth': 7, 'max_leaf_nodes': 10}


### Analyse des résultats
Afficher sur une courbe les scores de chacun des algorithmes avec la meilleure valeur d'hyperparamètre possible sur l'échantillon de **test**.

In [ ]:
# A vous

Pour chacune des méthodes, pour chaque meilleur hyperparamètre, effectuer le test de McNemar sur l'échantillon test pour décider finalement quel est la meilleure solution pour la prédiction de la charge de chauffage, avec 95% de confiance. 

In [ ]:
# A vous

### Remise du résultat

Alors, quelle est la meilleure solution (algorithme et hyper-paramètres ) ? 

Et enfin, pour cette solution, quelle est le taux réel de bonne classification, estimé par validation croisée 10 folds ? 

In [ ]:
# a vous